In [1]:
import sys
sys.path.append("..")
import argparse
import numpy as np
import dgl
from dgl import DGLGraph
import torch
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from collections import Counter
import pickle
import h5py
import random
import glob2
import seaborn as sns

import train
import models

%load_ext autoreload
%autoreload 2

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = train.get_device()

Using backend: pytorch
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Detected an old version of PyTorch. Suggest using torch>=1.5.0 for the best experience.
  return warnings.warn(message, category=category, stacklevel=1)
/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [3]:
category = "real_data"

epochs = 10
batch_size = 128 
pca_size = 50
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
print(files)
nb_genes =2000

In [7]:
results = pd.DataFrame()
model_name = "GraphConv"
normalize_weights = "log_per_cell"
node_features = "scale"
same_edge_values = False
edge_norm = True
hidden_relu = False
hidden_bn = False
n_layers = 1
hidden_dim = 200
hidden = [300]
nb_genes = 3000
activation = F.relu
for dataset in files:
    print(f">> {dataset}")

    data_mat = h5py.File(f"{path}/real_data/{dataset}.h5", "r")

    for nb_genes in ["all", 500, 1000, 2000, 3000, 4000, 5000 ]:
         
        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        highly_genes=nb_genes
        if nb_genes == "all":
            highly_genes = X.shape[1]
        genes_idx, cells_idx = train.filter_data(X, highly_genes=highly_genes)
        X = X[cells_idx][:, genes_idx]
        Y = Y[cells_idx]
        n_clusters = len(np.unique(Y))

        t0 = time.time()
        graph = train.make_graph(
            X,
            Y,
            dense_dim=pca_size,
            node_features=node_features,
            normalize_weights=normalize_weights,
        )

        labels = graph.ndata["label"]
        train_ids = np.where(labels != -1)[0]

        sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers)

        dataloader = dgl.dataloading.NodeDataLoader(
            graph,
            train_ids,
            sampler,
            batch_size=batch_size,
            shuffle=True,
            drop_last=False,
            num_workers=1,
        )
        print(
            f"INPUT: {model_name}  {hidden_dim}, {hidden}, {same_edge_values}, {edge_norm}"
        )
        t1 = time.time()

        for run in range(3):
            t_start = time.time()
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)

            model = models.GCNAE(
                in_feats=pca_size,
                n_hidden=hidden_dim,
                n_layers=n_layers,
                activation=activation,
                dropout=0.1,
                hidden=hidden,
                hidden_relu=hidden_relu,
                hidden_bn=hidden_bn,
            ).to(device)
            if run == 0:
                print(f">", model)

            optim = torch.optim.Adam(model.parameters(), lr=1e-5)

            scores = train.train(model, optim, epochs, dataloader, n_clusters, plot=False,
                                cluster=["KMeans", "Leiden"])
            scores["dataset"] = dataset
            scores["run"] = run
            scores["nb_genes"] = nb_genes
            scores["hidden"] = str(hidden)
            scores["hidden_dim"] = str(hidden_dim)
            scores["tot_kmeans_time"] = (t1-t0) + (scores['ae_end'] - t_start) + scores['kmeans_time']
            scores["tot_leiden_time"] = (t1-t0) + (scores['ae_end'] - t_start) + scores['leiden_time']
            scores["time_graph"] = t1-t0
            scores["time_training"] = (scores['ae_end'] - t_start)

            results = results.append(scores, ignore_index = True)

            results.to_pickle(
                f"../output/pickle_results/{category}/{category}_nb_genes.pkl")
            print("Done")

>> Quake_Smart-seq2_Trachea


/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


INPUT: GraphConv  200, [300], False, True
> GCNAE(
  (dropout): Dropout(p=0.1, inplace=False)
  (layer1): WeightedGraphConv(in=50, out=200, normalization=both, activation=<function relu at 0x7fbb619c77a0>)
  (decoder): InnerProductDecoder()
  (encoder): Sequential(
    (0): Linear(in_features=200, out_features=300, bias=True)
  )
)



ARI 0.9013, 0.48331236839294434
Done



ARI 0.8616, 0.46073225140571594
Done


...

/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


INPUT: GraphConv  200, [300], False, True
> GCNAE(
  (dropout): Dropout(p=0.1, inplace=False)
  (layer1): WeightedGraphConv(in=50, out=200, normalization=both, activation=<function relu at 0x7fbb619c77a0>)
  (decoder): InnerProductDecoder()
  (encoder): Sequential(
    (0): Linear(in_features=200, out_features=300, bias=True)
  )
)



ARI 0.6763, 0.483496755361557
Done



ARI 0.68, 0.468085378408432
Done



ARI 0.6792, 0.4728746712207794
Done


In [8]:
results.groupby("nb_genes").mean()

,ae_end,kmeans_ari,kmeans_cal,kmeans_nmi,kmeans_sil,kmeans_time,leiden_ari,leiden_cal,leiden_nmi,leiden_sil,leiden_time,run,time_graph,time_training,tot_kmeans_time,tot_leiden_time
nb_genes,,,,,,,,,,,,,,,,
500,1.614941e+09,0.730429,1147.234957,0.769811,0.375808,8.210394,0.549724,781.443925,0.706331,0.269425,16.755522,1.0,0.673396,8.938118,17.821908,26.367036
1000,1.614941e+09,0.747642,1509.430997,0.788638,0.441104,8.125889,0.546589,983.221820,0.721533,0.300803,16.513003,1.0,0.808320,10.047905,18.982115,27.369229
2000,1.614941e+09,0.758622,1707.121223,0.803113,0.465377,8.165497,0.552327,1057.287316,0.731696,0.312346,15.719749,1.0,1.070062,13.289310,22.524869,30.079121
3000,1.614941e+09,0.778107,1763.814241,0.816500,0.476133,8.100571,0.521511,1093.620271,0.723858,0.300350,15.536466,1.0,1.338324,16.459364,25.898259,33.334155
4000,1.614941e+09,0.768302,1791.269699,0.809320,0.475603,8.096766,0.536267,1113.055554,0.728496,0.303446,15.538731,1.0,1.606304,18.920053,28.623123,36.065088
5000,1.614941e+09,0.768778,1792.741781,0.809371,0.469368,8.086422,0.515284,1104.675844,0.723822,0.292064,15.412826,1.0,1.919229,22.489791,32.495442,39.821846
all,1.614940e+09,0.724998,1958.521541,0.786624,0.419590,8.408961,0.504473,1195.885809,0.714856,0.270978,15.634772,1.0,5.538507,78.475788,92.423255,99.649067


In [7]:
results = pd.DataFrame()
model_name = "GraphConv"
normalize_weights = "log_per_cell"
node_features = "scale"
same_edge_values = False
edge_norm = True
hidden_relu = False
hidden_bn = False
n_layers = 1
hidden_dim = 200
hidden = [300]
nb_genes = 3000
activation = F.relu
for dataset in files:
    print(f">> {dataset}")

    data_mat = h5py.File(f"{path}/real_data/{dataset}.h5", "r")

    Y = np.array(data_mat['Y'])
    X = np.array(data_mat['X'])

#     genes_idx, cells_idx = train.filter_data(X, highly_genes=nb_genes)
#     X = X[cells_idx][:, genes_idx]
#     Y = Y[cells_idx]

    for train_size in [ 1, 0.75, 0.5, 0.25
                                            ]:
        for run in range(3):
            n_clusters = len(np.unique(Y))
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            if train_size == 1:
                X_train = X
                y_train = Y
            else:
                X_train, X_test, y_train, y_test = train_test_split(
                    X, Y, train_size=train_size, random_state=run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            
            genes_idx, cells_idx = train.filter_data(X_train, highly_genes=nb_genes)
            X_train = X_train[cells_idx][:, genes_idx]
            y_train = y_train[cells_idx]
            
            graph = train.make_graph(
                X_train,
                y_train,
                dense_dim=pca_size,
                node_features=node_features,
                normalize_weights=normalize_weights,
            )

            labels = graph.ndata["label"]
            train_ids = np.where(labels != -1)[0]

            sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers)

            dataloader = dgl.dataloading.NodeDataLoader(
                graph,
                train_ids,
                sampler,
                batch_size=batch_size,
                shuffle=True,
                drop_last=False,
                num_workers=1,
            )
            print(
                f"INPUT: {model_name}  {hidden_dim}, {hidden}, {same_edge_values}, {edge_norm}"
            )
            t1 = time.time()

            t_start = time.time()
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)

            model = models.GCNAE(
                in_feats=pca_size,
                n_hidden=hidden_dim,
                n_layers=n_layers,
                activation=activation,
                dropout=0.1,
                hidden=hidden,
                hidden_relu=hidden_relu,
                hidden_bn=hidden_bn,
            ).to(device)
            if run == 0:
                print(f">", model)

            optim = torch.optim.Adam(model.parameters(), lr=1e-5)

            scores = train.train(model, optim, epochs, dataloader, n_clusters, plot=False,
                                cluster=["KMeans", "Leiden"])
            scores["dataset"] = dataset
            scores["run"] = run
            scores["nb_genes"] = nb_genes
            scores["hidden"] = str(hidden)
            scores["train_size"] = train_size


            results = results.append(scores, ignore_index = True)

            results.to_pickle(
                f"../output/pickle_results/{category}/{category}_nb_cells.pkl")
            print("Done")

>> Quake_Smart-seq2_Trachea


/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


INPUT: GraphConv  200, [300], False, True
> GCNAE(
  (dropout): Dropout(p=0.1, inplace=False)
  (layer1): WeightedGraphConv(in=50, out=200, normalization=both, activation=<function relu at 0x7f815e5af7a0>)
  (decoder): InnerProductDecoder()
  (encoder): Sequential(
    (0): Linear(in_features=200, out_features=300, bias=True)
  )
)



ARI 0.8732, 0.46117594838142395
Done


/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


...

INPUT: GraphConv  200, [300], False, True



ARI 0.6966, 0.38713857531547546
Done


/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


INPUT: GraphConv  200, [300], False, True



ARI 0.6233, 0.32860496640205383
Done


In [8]:
results.groupby("train_size").mean()

,ae_end,kmeans_ari,kmeans_cal,kmeans_nmi,kmeans_sil,kmeans_time,leiden_ari,leiden_cal,leiden_nmi,leiden_sil,leiden_time,nb_genes,run
train_size,,,,,,,,,,,,,
0.25,1.614961e+09,0.750793,339.771521,0.795131,0.389796,1.581852,0.621711,240.288876,0.746909,0.255558,2.867400,3000.0,1.0
0.50,1.614961e+09,0.770291,865.851119,0.812764,0.446581,3.435573,0.578191,549.270195,0.739573,0.293834,6.275479,3000.0,1.0
0.75,1.614961e+09,0.775329,1372.817428,0.813196,0.470222,5.720554,0.553296,885.591429,0.732751,0.299972,10.231872,3000.0,1.0
1.00,1.614961e+09,0.778107,1763.814247,0.816500,0.476133,7.719248,0.521516,1093.618419,0.723858,0.300338,15.338706,3000.0,1.0
